In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[2]))

,Net,Temprature
0,2078.060021,-5.9
1,2057.993265,-5.8
2,2039.019542,-5.8
3,2027.431312,-5.7
4,2026.809889,-5.7
...,...,...
43818,3637.456602,11.7
43819,3639.019312,12.3
43820,3639.463899,13.3
43821,3630.184859,13.2


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[2]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_FWEST,Temp_FWEST
0,2078.060021,-5.9
1,2078.060021,-5.9
2,2057.993265,-5.8
3,2039.019542,-5.8
4,2027.431312,-5.7
...,...,...
43819,3637.456602,11.7
43820,3639.019312,12.3
43821,3639.463899,13.3
43822,3630.184859,13.2


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_FWEST,Temp_FWEST,Lag-2,Lag-7
0,2131.053000,-2.3,1716.526120,2078.060021
1,2080.378240,-2.2,1685.924215,2078.060021
2,2066.741680,-2.3,1682.241812,2057.993265
3,2064.296987,-2.6,1688.457529,2039.019542
4,2073.546964,-3.1,1690.957125,2027.431312
...,...,...,...,...
43651,3637.456602,11.7,3791.082901,3649.932706
43652,3639.019312,12.3,3793.651872,3640.453277
43653,3639.463899,13.3,3795.557217,3624.437253
43654,3630.184859,13.2,3779.620366,3619.528769


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 23s 17ms/step - loss: 0.0088 - mape: 3282.8516 - mae: 0.0704 - val_loss: 0.0119 - val_mape: 9.5506 - val_mae: 0.0824
Epoch 2/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0049 - mape: 3046.4666 - mae: 0.0537 - val_loss: 0.0153 - val_mape: 12.7362 - val_mae: 0.1064
Epoch 3/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0037 - mape: 2670.0337 - mae: 0.0471 - val_loss: 0.0753 - val_mape: 31.8198 - val_mae: 0.2579
Epoch 4/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0033 - mape: 2658.6997 - mae: 0.0446 - val_loss: 0.0508 - val_mape: 26.8234 - val_mae: 0.2159
Epoch 5/100
1105/1105 [==============================] - 18s 17ms/step - loss: 0.0030 - mape: 2124.5444 - mae: 0.0426 - val_loss: 0.0397 - val_mape: 22.4387 - val_mae: 0.1835
Epoch 6/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0028 - mape: 2267.0847 - mae: 0.0409 - val_lo

1105/1105 [==============================] - 17s 16ms/step - loss: 0.0012 - mape: 1640.9563 - mae: 0.0260 - val_loss: 0.0593 - val_mape: 27.4124 - val_mae: 0.2236
Epoch 48/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0012 - mape: 1839.8993 - mae: 0.0261 - val_loss: 0.0511 - val_mape: 25.1691 - val_mae: 0.2064
Epoch 49/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 1272.7202 - mae: 0.0259 - val_loss: 0.0582 - val_mape: 26.4893 - val_mae: 0.2171
Epoch 50/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 1430.4379 - mae: 0.0257 - val_loss: 0.0498 - val_mape: 24.6239 - val_mae: 0.2016
Epoch 51/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0011 - mape: 1645.9089 - mae: 0.0253 - val_loss: 0.0395 - val_mape: 21.6542 - val_mae: 0.1777
Epoch 52/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0011 - mape: 1268.6025 - mae: 0.0251 - val_loss: 0.

Epoch 93/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.3219e-04 - mape: 1106.6215 - mae: 0.0206 - val_loss: 0.0521 - val_mape: 25.1942 - val_mae: 0.2054
Epoch 94/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.1077e-04 - mape: 1200.9221 - mae: 0.0203 - val_loss: 0.0526 - val_mape: 25.0758 - val_mae: 0.2051
Epoch 95/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.0449e-04 - mape: 1277.7438 - mae: 0.0202 - val_loss: 0.0444 - val_mape: 22.7526 - val_mae: 0.1865
Epoch 96/100
1105/1105 [==============================] - 18s 16ms/step - loss: 7.0081e-04 - mape: 1170.1425 - mae: 0.0202 - val_loss: 0.0589 - val_mape: 27.3531 - val_mae: 0.2222
Epoch 97/100
1105/1105 [==============================] - 17s 16ms/step - loss: 7.0024e-04 - mape: 1416.9839 - mae: 0.0202 - val_loss: 0.0479 - val_mape: 23.1685 - val_mae: 0.1910
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 7.1777e-04 - mape: 9

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1882.4671833515167


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 14ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.6021396 , 0.5743512 , 0.55194104, ..., 0.8064606 , 0.78765196,
       0.76443094], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.88354441, 0.86621942, 0.84593882, ..., 0.99218972, 0.98752933,
       0.98646656])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,3423.146738,2862.858643
1,3388.651995,2807.530762
2,3348.272503,2762.911133
3,3330.179385,2746.180908
4,3335.172114,2751.924805
...,...,...
4363,3637.456602,3287.982178
4364,3639.019312,3288.764648
4365,3639.463899,3269.669922
4366,3630.184859,3232.220947


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

14.371877748471526